# Fine-Tuning CodeLlama-7B for AT&T SQL Query Generation

## Approach: QLoRA (4-bit Quantization + LoRA Adapters)

This notebook fine-tunes **CodeLlama-7B-Instruct** to generate SQL queries for the AT&T telecom data warehouse schema.

### RAG vs Fine-Tuning — Two Approaches to the Same Problem

| Aspect | RAG (Previous Demo) | Fine-Tuning (This Demo) |
|--------|--------------------|--------------------------|
| Schema knowledge | Retrieved at query time from vector DB | Baked into model weights during training |
| Inference cost | Requires embedding + retrieval + LLM call | Single model inference (no retrieval step) |
| Schema changes | Just re-index — no retraining needed | Requires re-training on new schema |
| Model size | Uses large cloud LLM (GPT-4o) | Small 7B model, can run on-premise |
| Latency | Higher (retrieval + LLM) | Lower (single forward pass) |
| Best for | Frequently changing schemas | Stable schemas with high query volume |

### What is QLoRA?

**QLoRA** = Quantized Low-Rank Adaptation. It combines two techniques:

1. **4-bit Quantization**: Compresses the 7B parameter model from ~28GB → ~4GB in VRAM by storing weights in 4-bit precision
2. **LoRA (Low-Rank Adaptation)**: Instead of updating all 7B parameters, we inject small trainable adapter matrices into attention layers. Only ~1-2% of parameters are trained.

Result: We can fine-tune a 7B model on a **free Colab T4 GPU** (16GB VRAM).

### Prerequisites

- **Google Colab** with **T4 GPU** runtime (Runtime → Change runtime type → T4 GPU)
- **training_data.jsonl** file generated by `01_generate_training_data.py` on your local machine

## Step 1: Verify GPU & Install Dependencies

Make sure you have selected **T4 GPU** runtime before running this cell.

In [ ]:
# Verify GPU is available
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

In [ ]:
# Install required libraries
!pip install -q transformers==4.44.2 datasets accelerate==0.34.2 peft==0.13.0 bitsandbytes==0.44.0 trl==0.11.1

## Step 2: Upload Training Data

Upload the `training_data.jsonl` file that was generated on your local machine.

**To generate the training data locally:**
```bash
cd sql_generation_finetuning
python 01_generate_training_data.py
```

This produces `training_data/training_data.jsonl`. Upload that file below.

In [ ]:
from google.colab import files
import json

print("Please upload your training_data.jsonl file...")
uploaded = files.upload()

# Find the uploaded file
filename = list(uploaded.keys())[0]
print(f"\nUploaded: {filename} ({len(uploaded[filename])} bytes)")

# Preview the data
with open(filename, 'r') as f:
    lines = f.readlines()

print(f"Total training examples: {len(lines)}")
print("\n--- Sample entry ---")
sample = json.loads(lines[0])
print(f"Instruction: {sample['instruction'][:100]}...")
print(f"Input: {sample['input']}")
print(f"Output: {sample['output'][:200]}...")

## Step 3: Load Data into HuggingFace Dataset

In [ ]:
from datasets import load_dataset, Dataset

# Load JSONL into HuggingFace Dataset
dataset = load_dataset('json', data_files=filename, split='train')

print(f"Dataset size: {len(dataset)} examples")
print(f"Columns: {dataset.column_names}")
print(f"\nSample:")
print(dataset[0])

In [ ]:
# Split: 90% train, 10% validation
split = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split['train']
val_dataset = split['test']

print(f"Training examples: {len(train_dataset)}")
print(f"Validation examples: {len(val_dataset)}")

## Step 4: Load CodeLlama-7B with 4-bit Quantization

We load the model in 4-bit precision using **BitsAndBytes** NF4 quantization.
This reduces VRAM from ~28GB → ~4GB, making it fit on a T4 GPU.

**NF4 (NormalFloat4)** is a quantization scheme optimized for normally-distributed neural network weights.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_ID = "codellama/CodeLlama-7b-Instruct-hf"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                     # Load model weights in 4-bit
    bnb_4bit_quant_type="nf4",             # NormalFloat4 — best for pretrained weights
    bnb_4bit_compute_dtype=torch.float16,   # Compute in float16 for speed
    bnb_4bit_use_double_quant=True,         # Double quantization saves more memory
)

print(f"Loading tokenizer for {MODEL_ID}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token  # CodeLlama doesn't have a pad token
tokenizer.padding_side = "right"            # Pad on right for causal LM

print(f"Loading model in 4-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",                      # Automatically place on GPU
)

# Check memory usage
print(f"\nModel loaded successfully!")
print(f"GPU memory used: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"Model dtype: {model.dtype}")

## Step 5: Apply LoRA Adapters

**LoRA** injects small trainable matrices into the model's attention layers.
Instead of training all 7B parameters, we only train ~2M parameters (0.03%).

- **r=16**: Rank of the LoRA matrices (higher = more capacity, more VRAM)
- **lora_alpha=32**: Scaling factor (typically 2x the rank)
- **target_modules**: Which layers to add LoRA to (attention projections)

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare model for training with quantization
model = prepare_model_for_kbit_training(model)

# LoRA configuration
lora_config = LoraConfig(
    r=16,                                    # Rank of LoRA matrices
    lora_alpha=32,                           # Scaling factor
    target_modules=[                         # Apply to attention projection layers
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"  # Also MLP layers for better adaptation
    ],
    lora_dropout=0.05,                       # Small dropout for regularization
    bias="none",                             # Don't train bias terms
    task_type="CAUSAL_LM",                   # Causal language modeling task
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Print trainable parameter stats
model.print_trainable_parameters()
print(f"\nGPU memory after LoRA: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

## Step 6: Format Training Data for CodeLlama Instruct

CodeLlama Instruct uses a specific prompt format:
```
[INST] <<SYS>>
{system_message}
<</SYS>>

{user_message} [/INST]
{assistant_response}
```

We format each training example into this template so the model learns to respond in the expected way.

In [ ]:
def format_instruction(sample):
    """
    Convert an Alpaca-format sample into CodeLlama Instruct format.
    
    The model learns:
    - Everything inside [INST]...[/INST] is the prompt (not trained on)
    - Everything after [/INST] is what the model should generate (trained on)
    """
    system_msg = sample['instruction']
    user_msg = sample['input']
    assistant_msg = sample['output']

    return f"""<s>[INST] <<SYS>>
{system_msg}
<</SYS>>

{user_msg} [/INST]
{assistant_msg}</s>"""

# Preview a formatted example
print("--- Formatted training example ---")
print(format_instruction(train_dataset[0])[:500])
print("...")

# Check token lengths to ensure they fit in context window
sample_lengths = []
for sample in train_dataset:
    text = format_instruction(sample)
    tokens = tokenizer(text, return_tensors="pt")
    sample_lengths.append(tokens['input_ids'].shape[1])

print(f"\nToken length stats:")
print(f"  Min: {min(sample_lengths)}")
print(f"  Max: {max(sample_lengths)}")
print(f"  Mean: {sum(sample_lengths)/len(sample_lengths):.0f}")
print(f"  Examples > 2048 tokens: {sum(1 for l in sample_lengths if l > 2048)}")

## Step 7: Fine-Tune with SFTTrainer

**SFTTrainer** (Supervised Fine-Tuning Trainer) from the TRL library handles:
- Formatting examples using our template function
- Tokenization with proper padding/truncation
- Training loop with gradient accumulation
- Evaluation on validation set

**Training config rationale:**
- **3 epochs**: Enough to learn SQL patterns without overfitting
- **batch_size=2 × gradient_accumulation=4 = effective batch of 8**: Stable training on limited VRAM
- **learning_rate=2e-4**: Standard for LoRA fine-tuning
- **warmup_ratio=0.05**: Gentle warmup to avoid early training instability
- **fp16**: Mixed precision for speed (T4 supports FP16)

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Training arguments
training_args = TrainingArguments(
    output_dir="./sql-codellama-lora",       # Output directory for checkpoints
    num_train_epochs=3,                       # Number of training epochs
    per_device_train_batch_size=2,            # Batch size per GPU
    per_device_eval_batch_size=2,             # Eval batch size
    gradient_accumulation_steps=4,            # Effective batch = 2 * 4 = 8
    learning_rate=2e-4,                       # Learning rate for LoRA
    lr_scheduler_type="cosine",               # Cosine annealing schedule
    warmup_ratio=0.05,                        # 5% warmup steps
    fp16=True,                                # Mixed precision training
    logging_steps=10,                         # Log every 10 steps
    eval_strategy="epoch",                    # Evaluate at end of each epoch
    save_strategy="epoch",                    # Save checkpoint each epoch
    load_best_model_at_end=True,              # Load best model when done
    report_to="none",                         # No external logging (wandb etc.)
    optim="paged_adamw_8bit",                 # 8-bit AdamW to save memory
    max_grad_norm=0.3,                        # Gradient clipping
    weight_decay=0.001,                       # Small weight decay
)

# Create trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    formatting_func=format_instruction,        # Our custom formatting function
    max_seq_length=2048,                       # Max sequence length
    packing=False,                             # Don't pack multiple examples
)

print("Trainer created. Starting fine-tuning...")
print(f"Training examples: {len(train_dataset)}")
print(f"Validation examples: {len(val_dataset)}")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
estimated_steps = (len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)) * training_args.num_train_epochs
print(f"Estimated total steps: ~{estimated_steps}")

In [ ]:
# Train!
train_result = trainer.train()

# Print training metrics
print("\n" + "=" * 50)
print("TRAINING COMPLETE")
print("=" * 50)
print(f"Training loss: {train_result.training_loss:.4f}")
print(f"Training runtime: {train_result.metrics['train_runtime']:.0f} seconds")
print(f"GPU memory peak: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")

## Step 8: Test the Fine-Tuned Model

Now let's test with the same sample questions from the RAG demo to compare outputs.

The model should generate SQL that:
- Uses only tables/columns from the AT&T schema
- Has correct JOINs and relationships
- Uses CTEs, window functions, and aggregations where appropriate

In [ ]:
def generate_sql(question, max_new_tokens=1024):
    """
    Generate SQL from a natural language question using the fine-tuned model.
    """
    system_msg = (
        "You are an expert SQL developer for AT&T's enterprise telecom data warehouse. "
        "Write a precise, production-quality SQL query (T-SQL / SQL Server syntax) "
        "for the given business requirement. Use CTEs, window functions, proper JOINs, "
        "and clear formatting as appropriate."
    )

    prompt = f"""<s>[INST] <<SYS>>
{system_msg}
<</SYS>>

{question} [/INST]
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,         # Greedy decoding for deterministic SQL
            temperature=1.0,
            repetition_penalty=1.1,  # Avoid repetition
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decode only the new tokens (skip the prompt)
    generated = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return generated.strip()


# Quick test
test_q = "List all active customers with their current service plan name and monthly charges"
print(f"Question: {test_q}\n")
sql = generate_sql(test_q)
print(f"Generated SQL:\n{sql}")

In [ ]:
# Test with all sample questions from the RAG demo

SAMPLE_QUESTIONS = [
    # Basic Joins
    "List all active customers with their current service plan name and monthly charges",
    "Show all equipment currently assigned to customers along with the customer name and device model",
    # Multi-Table Joins & Aggregation
    "Show total revenue by region for the last quarter, broken down by product category",
    "Find the top 10 customers by total payment amount in the last 12 months, including their account type and region",
    # Subqueries & Complex Filters
    "Find customers who have overdue invoices exceeding $500 and have also filed trouble tickets in the past 30 days",
    "List vendors whose contract is expiring within 90 days, along with their total purchase order value and number of active equipment units they supplied",
    # CTEs & Window Functions
    "Generate a monthly revenue trend report with month-over-month growth percentage for each product category over the last 12 months",
    "Show customer churn analysis: customers who cancelled subscriptions in the last 6 months with their lifetime value, average monthly bill, and last trouble ticket reason",
    # Advanced Analytics
    "Identify the top 10 cell towers by total data usage volume and show their region, number of connected customers, technology type, and average signal quality",
    "Create a comprehensive billing reconciliation report showing invoices where the total payments received do not match the invoice amount, including customer name, payment method, days overdue, and outstanding balance",
    "Rank regions by a composite customer satisfaction score derived from average trouble ticket resolution time in hours, billing dispute frequency per 1000 customers, and average network uptime percentage across cell towers in each region",
]

print("=" * 70)
print("FINE-TUNED MODEL — SQL GENERATION RESULTS")
print("=" * 70)

for i, question in enumerate(SAMPLE_QUESTIONS, 1):
    print(f"\n{'='*70}")
    print(f"Question {i}: {question}")
    print(f"{'='*70}")
    sql = generate_sql(question)
    print(sql)
    print()

## Step 9: Save the Fine-Tuned Model

We save only the **LoRA adapters** (a few MB), not the full model.
To use later, you load the base model + adapters and merge them.

In [ ]:
# Save the LoRA adapters
ADAPTER_DIR = "./sql-codellama-lora-final"
model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(ADAPTER_DIR)

print(f"LoRA adapters saved to {ADAPTER_DIR}")

# Show what was saved
import os
for f in os.listdir(ADAPTER_DIR):
    size = os.path.getsize(os.path.join(ADAPTER_DIR, f))
    print(f"  {f}: {size / 1e6:.2f} MB")

In [ ]:
# Download the adapters to your local machine
import shutil

# Zip the adapter directory
shutil.make_archive("sql-codellama-lora-final", 'zip', ADAPTER_DIR)

# Download
from google.colab import files
files.download("sql-codellama-lora-final.zip")
print("Download complete! This zip contains your LoRA adapters.")

## Step 10: How to Load the Saved Model Later

To use the fine-tuned model in production or another notebook:

In [ ]:
# -- This cell shows how to reload the model later --
# -- You don't need to run this now --

'''
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

# 1. Load base model with quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
base_model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-7b-Instruct-hf",
    quantization_config=bnb_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")

# 2. Load LoRA adapters on top
model = PeftModel.from_pretrained(base_model, "./sql-codellama-lora-final")

# 3. Now use generate_sql() as before
'''

print("See code above for reloading the model with saved LoRA adapters.")

---

## Summary

In this notebook we:

1. **Uploaded** ~350 synthetic question-SQL training pairs (generated locally using Azure OpenAI)
2. **Loaded** CodeLlama-7B-Instruct in 4-bit quantization (~4GB VRAM)
3. **Applied** LoRA adapters to attention + MLP layers (~2M trainable params)
4. **Fine-tuned** for 3 epochs using SFTTrainer with CodeLlama's instruct template
5. **Tested** with 11 sample questions ranging from simple joins to complex CTEs + window functions
6. **Saved** the LoRA adapters (few MB) for deployment

### Key Takeaway

A fine-tuned 7B model can learn a specific database schema and generate valid SQL **without** needing RAG retrieval at inference time. The tradeoff: schema changes require re-training, but inference is faster and doesn't need a vector database.